<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/elegiveis_cb_fees/elegiveis_cb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
import numpy as np

ModuleNotFoundError: No module named 'xlsxwriter'

In [5]:
!pip install dask[dataframe] pyxlsb

In [6]:
#especificar nome dos ficheiros

resumo_user_name="P_Resumo_users_setembro23.xlsb"
apoio_name="Apoio_Base_Proactiva.xlsb"

In [7]:
#importar ficheiros - importar resumo_user e apoio

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving Apoio_Base_Proactiva.xlsb to Apoio_Base_Proactiva.xlsb


Saving P_Resumo_users_setembro23.xlsb to P_Resumo_users_setembro23.xlsb


In [8]:
#ler dataframes
resumo_user=pd.read_excel(resumo_user_name, sheet_name=0)

apoio_base_sr=pd.read_excel(apoio_name, sheet_name="SR")
apoio_base_ativacao=pd.read_excel(apoio_name, sheet_name="Ativação")
apoio_base_gal=pd.read_excel(apoio_name, sheet_name="GAL-1")
apoio_base_portabilidade=pd.read_excel(apoio_name, sheet_name="Portabilidades")

#substituir nomes das colunas no ficheiro resumo_user
resumo_user2=resumo_user.copy()

resumo_user2.columns=[i.replace("RRR", "").strip() if i.find("RRR")!=-1 else i for i in resumo_user.columns.tolist()]

In [ ]:
#apoio_base_ativacao2=apoio_base_ativacao.copy()



Retention Username
DVMONTEIRO            36
MAANSOARES            17
CPMEIRELES            16
JARISANTOS            15
DJLEITE               14
                      ..
MACONASCIMEN           1
CMARINEVES             1
KAREIS                 1
LIANSANTOS             1
FRSOCARVALHO           1
Name: count, Length: 119, dtype: int64

In [38]:
#sr -> coluna h (Elegivel IB (SR)) e l (Linhas CB a Zero IB)
#activação -> coluna k (Valor User IB) e coluna "service request id" do ficheiro gal n+1
#portabilidade -> coluna k (Valor User IB)

lista1=["Service Request ID", "Service Request ID", "Retention Username", "Purchase Order Item", "Retention Username"]
lista2=["Elegivel IB (SR)", "Linhas CB a Zero IB", "Valor User IB", "Purchase Order Item", "Valor User IB"]

for count, (i, j) in enumerate(zip(lista1, lista2)):

  if count==0:
    apoio_base_sr["Elegivel"]=apoio_base_sr[[i]].merge(right=resumo_user2, how="left", on=i) [j]

  elif count==1:
    apoio_base_sr["Valor Zero"]=apoio_base_sr[[i]].merge(right=resumo_user2, how="left", on=i) [j]


  elif count==2:
    #elegivel
    for user in apoio_base_ativacao[i].unique().tolist():
      indice=apoio_base_ativacao.loc[apoio_base_ativacao[i]==user, i].index.tolist()
      valor=resumo_user2.loc[resumo_user2["Retention UserName"]==user, j].unique().tolist()[0]
      if valor>0:
        apoio_base_ativacao.loc[indice, "Elegivel"]="s"
      else:
        apoio_base_ativacao.loc[indice, "Elegivel"]="n"


  elif count==3:
    for order in apoio_base_ativacao[i].unique().tolist():
      indice=apoio_base_ativacao.loc[apoio_base_ativacao[i]==order, i].index.tolist()
      if apoio_base_gal.loc[apoio_base_gal[i]==order, j].count()>0:
        valor=apoio_base_gal.loc[apoio_base_gal[i]==order, j].unique().tolist()[0]
        apoio_base_ativacao.loc[indice, "gal-1"]=valor
      else:
        apoio_base_ativacao.loc[indice, "gal-1"]="#N/A"

  else:
    for retention in apoio_base_portabilidade[i].unique().tolist():
      indice=apoio_base_portabilidade.loc[apoio_base_portabilidade[i]==retention, :].index
      valor=resumo_user2.loc[resumo_user2["Retention UserName"]==retention, j].unique().tolist() [0]
      if valor <=0:
        apoio_base_portabilidade.loc[indice, "Elegivel"]="n"
      else:
        apoio_base_portabilidade.loc[indice, "Elegivel"]="s"


In [39]:
apoio_base_ativacao.loc[:, "gal-1"]


0      #N/A
1      #N/A
2      #N/A
3      #N/A
4      #N/A
       ... 
583    #N/A
584    #N/A
585    #N/A
586    #N/A
587    #N/A
Name: gal-1, Length: 588, dtype: object

In [21]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00


In [40]:
excel_file_path = 'apoio_base_elegivel.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    apoio_base_sr.to_excel(writer, sheet_name='SR', index=False)
    apoio_base_ativacao.to_excel(writer, sheet_name='Ativação', index=False)
    apoio_base_gal.to_excel(writer, sheet_name='GAL-1', index=False)
    apoio_base_portabilidade.to_excel(writer, sheet_name='Portabilidades', index=False)

In [42]:
files.download('apoio_base_elegivel.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>